In [1]:
cd set3

/media/datastorage/Phong/ufc101/set3


In [2]:
ls -l

total 21970960
-rw-rw-r-- 1 bribeiro bribeiro        113 jan 25 10:37 checkpoint
-rw-rw-r-- 1 bribeiro bribeiro  138864586 jan 22 00:43 DenseNet201_CNNRNN2-set3.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 jan 22 00:43 DenseNet201_CNNRNN2-set3.index
-rw-rw-r-- 1 bribeiro bribeiro 2838897848 jan 19 21:41 DenseNet201_CNNRNN_test_100_set3.pickle
-rw-rw-r-- 1 bribeiro bribeiro      29696 jan 19 21:41 DenseNet201_CNNRNN_test_labels_set3.npy
-rw-rw-r-- 1 bribeiro bribeiro 7392194652 jan 19 21:36 DenseNet201_CNNRNN_train_100_set3.pickle
-rw-rw-r-- 1 bribeiro bribeiro      77120 jan 19 21:41 DenseNet201_CNNRNN_train_labels_set3.npy
-rw-rw-r-- 1 bribeiro bribeiro 1070705996 jan 25 10:37 DenseNet201_Transformer-set3.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro       4595 jan 25 10:37 DenseNet201_Transformer-set3.index
-rw-rw-r-- 1 bribeiro bribeiro  143058900 jan 21 23:50 ResNet152_CNNRNN2-set3.data-00000-of-00001
-rw-rw-r-- 1 bribeiro bribeiro        836 jan 21 23:50 ResN

In [3]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [4]:
IMG_SIZE = 224
# BATCH_SIZE = 32
EPOCHS = 20

MAX_SEQ_LENGTH = 100
NUM_FEATURES = 2048

In [3]:
# MAX_SEQ_LENGTH = 20
# NUM_FEATURES = 2048
# IMG_SIZE = 128

# EPOCHS = 10

In [5]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 9624
Total videos for testing: 3696


,video_name,tag
8599,v_TableTennisShot_g24_c06.avi,TableTennisShot
7872,v_Skijet_g24_c01.avi,Skijet
8256,v_SumoWrestling_g04_c02.avi,SumoWrestling
3690,v_HeadMassage_g02_c03.avi,HeadMassage
4321,v_JavelinThrow_g07_c01.avi,JavelinThrow
3824,v_HighJump_g07_c05.avi,HighJump
4935,v_LongJump_g23_c02.avi,LongJump
2499,v_Diving_g11_c03.avi,Diving
7066,v_RockClimbingIndoor_g10_c03.avi,RockClimbingIndoor
2751,v_FieldHockeyPenalty_g03_c01.avi,FieldHockeyPenalty


In [5]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


In [6]:

def build_feature_extractor():
    feature_extractor = keras.applications.ResNet152(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.resnet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

2023-01-21 03:18:40.261279: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 03:18:41.188769: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1


In [6]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'HammerThrow', 'Hammering', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpRope', 'JumpingJack', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

2023-01-25 11:18:23.726442: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 11:18:24.347995: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


In [7]:
from tqdm import tqdm

def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in tqdm(enumerate(video_paths)):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


# train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

# print(f"Frame features in train set: {train_data[0].shape}")
# print(f"Frame masks in train set: {train_data[1].shape}")

0it [00:00, ?it/s]2022-07-10 00:08:42.919387: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-07-10 00:08:43.148367: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:08:43.148778: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:08:43.148815: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-07-10 00:08:43.149460: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:08:43.149574: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
3783it [6:55:47,  6.59s/it]


In [9]:
train_data, train_labels = prepare_all_videos(train_df, "train")

2999it [5:29:26,  5.61s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

9537it [17:48:27,  6.72s/it]


In [ ]:
# # Utility for our sequence model.
# # Bidirectional

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5))(#16, 512
#         frame_features_input, mask=mask_input
#     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.LeakyReLU()(x)
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
#     return rnn_model


# # Utility for running experiments.
# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    
#     history = seq_model.fit(
#         [train_data[0], train_data[1]],
#         train_labels,
#         validation_split=0.2,
#         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# _, sequence_model = run_experiment()

In [ ]:
#40.13% LSTM(2048,dropout=0.1)
#46.47% LSTM(2048,dropout=0.5)
#43.96% LSTM(4096,dropout=0.5)

In [ ]:
# class_vocab = label_processor.get_vocabulary()

# frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
# mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# # Refer to the following tutorial to understand the significance of using `mask`:
# # https://keras.io/api/layers/recurrent_layers/gru/
# x = keras.layers.LSTM(2048, return_sequences=True,dropout=0.1)(#16, 512
#     frame_features_input, mask=mask_input
# )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
# x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
# output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

# rnn_model = keras.Model([frame_features_input, mask_input], output)

# rnn_model.compile(
#     loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# )

# print(rnn_model.summary())

In [ ]:
# class_vocab = label_processor.get_vocabulary()

# frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
# mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# # Refer to the following tutorial to understand the significance of using `mask`:
# # https://keras.io/api/layers/recurrent_layers/gru/
# x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.1)(#16, 512
#     frame_features_input, mask=mask_input
# )
# # x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
# x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
# output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

# rnn_model = keras.Model([frame_features_input, mask_input], output)

# rnn_model.compile(
#     loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# )

# print(rnn_model.summary())

In [ ]:

# def prepare_single_video(frames):
#     frames = frames[None, ...]
#     frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#     frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

#     for i, batch in enumerate(frames):
#         video_length = batch.shape[0]
#         length = min(MAX_SEQ_LENGTH, video_length)
#         for j in range(length):
#             frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
#         frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#     return frame_features, frame_mask


# def sequence_prediction(path):
#     class_vocab = label_processor.get_vocabulary()

#     frames = load_video(os.path.join("test", path))
#     frame_features, frame_mask = prepare_single_video(frames)
#     probabilities = sequence_model.predict([frame_features, frame_mask])[0]

#     for i in np.argsort(probabilities)[::-1]:
#         print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
#     return frames


# # This utility is for visualization.
# # Referenced from:
# # https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


# test_video = np.random.choice(test_df["video_name"].values.tolist())
# print(f"Test video path: {test_video}")
# test_frames = sequence_prediction(test_video)
# to_gif(test_frames[:MAX_SEQ_LENGTH])

In [ ]:
ls -l

In [ ]:
import numpy as np
import tensorflow as tf 

np.save('Inception_CNNRNN_train_data.npy', train_data)    # .npy extension is added if not given
# np.save('Inception_CNNRNN_test_data.npy', test_data[0])    # .npy extension is added if not given
# np.save('Inception_CNNRNN_train_labels.npy', train_data[1])    # .npy extension is added if not given
# np.save('Inception_CNNRNN_test_labels.npy', test_labels)    # .npy extension is added if not given

In [ ]:
print(train_data[0].shape)
print(train_data[1].shape)

In [ ]:
print(train_data.type)

In [10]:
import pickle

with open('Resnet152_CNNRNN_train_100.pickle', 'wb') as f:
    pickle.dump(train_data, f)

In [8]:
import pickle

with open('Resnet152_CNNRNN_test_100.pickle', 'wb') as f:
    pickle.dump(test_data, f)

In [ ]:
import numpy as np
import tensorflow as tf 

# np.save('densenet_train_data.npy', train_data)    # .npy extension is added if not given
# np.save('densenet_test_data.npy', test_data)    # .npy extension is added if not given
np.save('CNNRNN_train_labels.npy', train_labels)    # .npy extension is added if not given
np.save('CNNRNN_test_labels.npy', test_labels)    # .npy extension is added if not given

In [ ]:
ls -l

In [ ]:
mv Inception_CNNRNN_20.pickle Inception_CNNRNN_train_20.pickle

In [8]:
# import numpy as np
# import tensorflow as tf 
# import pickle

# with open('Resnet152_CNNRNN_train_100.pickle', 'rb') as f:
#      d_train_data = pickle.load(f)
# with open('Resnet152_CNNRNN_test_100.pickle', 'rb') as f:
#      d_test_data = pickle.load(f)
        
# # d_train_data = np.load('densenet_train_data.npy')
# # d_test_data = np.load('densenet_test_data.npy')
# d_train_labels = np.load('CNNRNN_train_labels.npy')
# d_test_labels = np.load('CNNRNN_test_labels.npy')

In [ ]:
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
# )
# # print(label_processor.get_vocabulary())

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
#         frame_features_input, mask=mask_input
#     )    
# #     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
# #         frame_features_input, mask=mask_input
# #     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.GaussianNoise(0.4)(x)
#     x = keras.layers.LeakyReLU(0.1)(x)    
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

# #     rnn_model.compile(
# #         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# #     )
#     return rnn_model

# # Utility for running experiments.
# def d_run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    
#     optimizer = keras.optimizers.SGD(lr=1e-3)
# #     optimizer = keras.optimizers.Adadelta()
    
#     seq_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
#     )    
    
#     history = seq_model.fit(
#         [d_train_data[0], d_train_data[1]],
#         d_train_labels,
# #         validation_split=0.2,
#         validation_data=([d_test_data[0], d_test_data[1]],
#         d_test_labels),
#         epochs=100,
# #         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# d_run_experiment()

In [ ]:
#lr=0.1 77.40%
#lr=0.01 78.69%
#lr=0.001 77.85%

In [ ]:
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
# )
# # print(label_processor.get_vocabulary())

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
#         frame_features_input, mask=mask_input
#     )    
# #     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
# #         frame_features_input, mask=mask_input
# #     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.GaussianNoise(0.4)(x)
#     x = keras.layers.LeakyReLU(0.1)(x)    
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

# #     rnn_model.compile(
# #         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# #     )
#     return rnn_model

# # Utility for running experiments.
# def d_run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    

#     lr_schedule = keras.callbacks.LearningRateScheduler(
#                   lambda epoch: 1e-6 * 10**(4*epoch / 10))
    
#     optimizer = keras.optimizers.SGD(lr=1e-3)
    
#     seq_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
#     )    
# #     seq_model.compile(optimizer=optimizer,
# #                       loss='categorical_crossentropy',
# #                      metrics=['accuracy'])     
    
#     history = seq_model.fit(
#         [d_train_data[0], d_train_data[1]],
#         d_train_labels,
# #         validation_split=0.2,
#         validation_data=([d_test_data[0], d_test_data[1]],
#         d_test_labels),
#         epochs=20,
# #         epochs=EPOCHS,
#         callbacks=[lr_schedule,checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# his, seq = d_run_experiment()


In [ ]:
from matplotlib import pyplot as plt

plt.semilogx(his.history['lr'], his.history['accuracy'])
plt.axis([1e-6, 1, 0, 1])
plt.xlabel('lr')
plt.ylabel('accuracy')
plt.show()

In [6]:
# import numpy as np

# MAX_SEQ_LENGTH = 20#override max sequence length
# #max_seq=40
# # copy_indices=[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38]
# # copy_indices=[0,4,8,12,16,20,24,28,32,36]
# # copy_indices=[0,5,10,15,20,25,30,35]
# # copy_indices=[0,8,16,24,32]
# # copy_indices=[0,19]
# # copy_indices=[0]
# # max_seq=100
# # # copy_indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
# #               26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
# #               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,
# #               76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
# copy_indices=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
# # copy_indices=[0,10,20,30,40,50,60,70,80,90]
# #copy_indices=[0,20,40,60,80]
# # copy_indices=[0,25,50,75]
# # copy_indices=[0,50]
# d_train_data2_1 = np.copy(d_train_data[0][:,copy_indices, ])
# d_train_data2_2 = np.copy(d_train_data[1][:,copy_indices, ])
# d_train_data2 = (d_train_data2_1,d_train_data2_2)

# d_test_data2_1 = np.copy(d_test_data[0][:,copy_indices, ])
# d_test_data2_2 = np.copy(d_test_data[1][:,copy_indices, ])
# d_test_data2 = (d_test_data2_1,d_test_data2_2)
# # print(d_train_data2.shape)

# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
# )
# # print(label_processor.get_vocabulary())

# def get_sequence_model():
#     class_vocab = label_processor.get_vocabulary()

#     frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
#     mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

#     # Refer to the following tutorial to understand the significance of using `mask`:
#     # https://keras.io/api/layers/recurrent_layers/gru/
#     x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
#         frame_features_input, mask=mask_input
#     )    
# #     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
# #         frame_features_input, mask=mask_input
# #     )
# #     x = keras.layers.LSTM(2048)(x)#8, 256
# #     x = keras.layers.Dropout(0.1)(x)
#     x = keras.layers.Dense(1024)(x)#8, 256
#     x = keras.layers.GaussianNoise(0.4)(x)
#     x = keras.layers.LeakyReLU(0.1)(x)    
#     output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

#     rnn_model = keras.Model([frame_features_input, mask_input], output)

# #     rnn_model.compile(
# #         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
# #     )
#     return rnn_model

# # Utility for running experiments.
# def d_run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     seq_model = get_sequence_model()
#     print(seq_model.summary())
    
#     optimizer = keras.optimizers.SGD(lr=1e-2)
# #     optimizer = keras.optimizers.Adadelta()
    
#     seq_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
#     )    
    
#     history = seq_model.fit(
#         [d_train_data2[0], d_train_data2[1]],
#         d_train_labels,
# #         validation_split=0.2,
#         validation_data=([d_test_data2[0], d_test_data2[1]],
#         d_test_labels),
#         epochs=100,
# #         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     seq_model.load_weights(filepath)
#     _, accuracy = seq_model.evaluate([d_test_data2[0], d_test_data2[1]], d_test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return history, seq_model


# d_run_experiment()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 20, 2048)]   0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 2048)         33562624    ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 dense_2 (Dense)                (None, 1024)         2098176     ['lstm_1[0][0]']           

2022-07-11 02:21:40.196429: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1562542080 exceeds 10% of free system memory.
2022-07-11 02:21:41.123292: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1562542080 exceeds 10% of free system memory.


Epoch 1/100
299/299 [==============================] - ETA: 0s - loss: 3.0337 - accuracy: 0.3914
Epoch 1: val_loss improved from inf to 3.90117, saving model to /tmp/video_classifier
299/299 [==============================] - 16s 45ms/step - loss: 3.0337 - accuracy: 0.3914 - val_loss: 3.9012 - val_accuracy: 0.2411
Epoch 2/100
299/299 [==============================] - ETA: 0s - loss: 0.8427 - accuracy: 0.8183
Epoch 2: val_loss did not improve from 3.90117
299/299 [==============================] - 11s 37ms/step - loss: 0.8427 - accuracy: 0.8183 - val_loss: 7.5004 - val_accuracy: 0.0603
Epoch 3/100
299/299 [==============================] - ETA: 0s - loss: 0.4036 - accuracy: 0.9182
Epoch 3: val_loss did not improve from 3.90117
299/299 [==============================] - 11s 37ms/step - loss: 0.4036 - accuracy: 0.9182 - val_loss: 4.0753 - val_accuracy: 0.3347
Epoch 4/100
299/299 [==============================] - ETA: 0s - loss: 0.2265 - accuracy: 0.9571
Epoch 4: val_loss improved from 3

Epoch 30/100
299/299 [==============================] - ETA: 0s - loss: 0.0032 - accuracy: 1.0000
Epoch 30: val_loss did not improve from 0.74427
299/299 [==============================] - 11s 38ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.7820 - val_accuracy: 0.8076
Epoch 31/100
299/299 [==============================] - ETA: 0s - loss: 0.0031 - accuracy: 1.0000
Epoch 31: val_loss did not improve from 0.74427
299/299 [==============================] - 11s 38ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.7809 - val_accuracy: 0.8084
Epoch 32/100
299/299 [==============================] - ETA: 0s - loss: 0.0029 - accuracy: 1.0000
Epoch 32: val_loss did not improve from 0.74427
299/299 [==============================] - 11s 38ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.7809 - val_accuracy: 0.8039
Epoch 33/100
299/299 [==============================] - ETA: 0s - loss: 0.0029 - accuracy: 1.0000
Epoch 33: val_loss did not improve from 0.74427
299/299 [========

299/299 [==============================] - 11s 38ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.8060 - val_accuracy: 0.8076
Epoch 60/100
299/299 [==============================] - ETA: 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 60: val_loss did not improve from 0.74427
299/299 [==============================] - 11s 38ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.8052 - val_accuracy: 0.8086
Epoch 61/100
299/299 [==============================] - ETA: 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 61: val_loss did not improve from 0.74427
299/299 [==============================] - 11s 38ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.8047 - val_accuracy: 0.8097
Epoch 62/100
299/299 [==============================] - ETA: 0s - loss: 0.0013 - accuracy: 1.0000
Epoch 62: val_loss did not improve from 0.74427
299/299 [==============================] - 11s 38ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.8056 - val_accuracy: 0.8097
Epoch 63/100
299/299 [========

Epoch 89/100
299/299 [==============================] - ETA: 0s - loss: 8.1803e-04 - accuracy: 1.0000
Epoch 89: val_loss did not improve from 0.74427
299/299 [==============================] - 11s 38ms/step - loss: 8.1803e-04 - accuracy: 1.0000 - val_loss: 0.8176 - val_accuracy: 0.8099
Epoch 90/100
299/299 [==============================] - ETA: 0s - loss: 8.2849e-04 - accuracy: 1.0000
Epoch 90: val_loss did not improve from 0.74427
299/299 [==============================] - 12s 39ms/step - loss: 8.2849e-04 - accuracy: 1.0000 - val_loss: 0.8204 - val_accuracy: 0.8089
Epoch 91/100
299/299 [==============================] - ETA: 0s - loss: 8.3624e-04 - accuracy: 1.0000
Epoch 91: val_loss did not improve from 0.74427
299/299 [==============================] - 12s 39ms/step - loss: 8.3624e-04 - accuracy: 1.0000 - val_loss: 0.8244 - val_accuracy: 0.8091
Epoch 92/100
299/299 [==============================] - ETA: 0s - loss: 7.7358e-04 - accuracy: 1.0000
Epoch 92: val_loss did not improve fr

ValueError: Shapes (2048, 8192) and (1920, 8192) are incompatible

In [ ]:
#max_seq=40
#seq=1 75.76%
#seq=2 78.48%
#seq=5 79.96%
#seq=8 79.88%
#seq=10 79.62%
#seq=20 79.62%

#max_seq=100
#seq=2 78.91%
#seq=4 80.89
#seq=5 80.31
#seq=10 81.07
#seq=20 81.18

In [7]:
import numpy as np
import tensorflow as tf 
import pickle

with open('Resnet152_CNNRNN_train_100_set3.pickle', 'rb') as f:
     d_train_data = pickle.load(f)
with open('Resnet152_CNNRNN_test_100_set3.pickle', 'rb') as f:
     d_test_data = pickle.load(f)
        
# d_train_data = np.load('densenet_train_data.npy')
# d_test_data = np.load('densenet_test_data.npy')
d_train_labels = np.load('Resnet152_CNNRNN_train_labels_set3.npy')
d_test_labels = np.load('Resnet152_CNNRNN_test_labels_set3.npy')

In [8]:
train_labels = d_train_labels
test_labels = d_test_labels

# print(f"Frame features in train set: {train_data.shape}")

In [20]:
import numpy as np

MAX_SEQ_LENGTH = 2#override max sequence length
#max_seq=40
# copy_indices=[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38]
# copy_indices=[0,4,8,12,16,20,24,28,32,36]
# copy_indices=[0,5,10,15,20,25,30,35]
# copy_indices=[0,8,16,24,32]
# copy_indices=[0,19]
# copy_indices=[0]
# max_seq=100
# # copy_indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
#               26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
#               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,
#               76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
# copy_indices=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
# copy_indices=[0,10,20,30,40,50,60,70,80,90]
#copy_indices=[0,20,40,60,80]
# copy_indices=[0,25,50,75]
copy_indices=[0,50]
train_data = np.copy(d_train_data[0][:,copy_indices, ])
test_data = np.copy(d_test_data[0][:,copy_indices, ])
print(train_data.shape)

(9624, 2, 2048)


In [10]:

class PositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask


In [11]:

class TransformerEncoder(keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [keras.layers.Dense(dense_dim, activation=tf.nn.gelu), keras.layers.Dense(embed_dim),]
        )
        self.layernorm_1 = keras.layers.LayerNormalization()
        self.layernorm_2 = keras.layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


In [24]:
# Label preprocessing with StringLookup.
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
)

def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 128
    num_heads = 6
    classes = len(label_processor.get_vocabulary())

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = "ResNet152_Transformer-set2"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    model = get_compiled_model()
    history = model.fit(
        train_data,
        train_labels,
        # validation_split=0.15,
        validation_data=([test_data, test_labels]),
        epochs=100,
        callbacks=[checkpoint],
    )
    
    print(f"Test best accuracy: {round(max(history.history['val_accuracy'])*100,2)}%")
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     model.load_weights(filepath)
#     _, accuracy = model.evaluate(test_data, test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

trained_model = run_experiment()

Epoch 1/100
301/301 [==============================] - ETA: 0s - loss: 5.8180 - accuracy: 0.0219
Epoch 1: val_loss improved from inf to 3.67794, saving model to ResNet152_Transformer-set2
301/301 [==============================] - 17s 53ms/step - loss: 5.8180 - accuracy: 0.0219 - val_loss: 3.6779 - val_accuracy: 0.1318
Epoch 2/100
299/301 [============================>.] - ETA: 0s - loss: 2.1306 - accuracy: 0.4764
Epoch 2: val_loss improved from 3.67794 to 1.69387, saving model to ResNet152_Transformer-set2
301/301 [==============================] - 16s 52ms/step - loss: 2.1230 - accuracy: 0.4781 - val_loss: 1.6939 - val_accuracy: 0.6117
Epoch 3/100
301/301 [==============================] - ETA: 0s - loss: 0.7940 - accuracy: 0.7692
Epoch 3: val_loss improved from 1.69387 to 1.41138, saving model to ResNet152_Transformer-set2
301/301 [==============================] - 16s 52ms/step - loss: 0.7940 - accuracy: 0.7692 - val_loss: 1.4114 - val_accuracy: 0.6721
Epoch 4/100
301/301 [========

Epoch 30/100
301/301 [==============================] - ETA: 0s - loss: 0.0461 - accuracy: 0.9855
Epoch 30: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 29ms/step - loss: 0.0461 - accuracy: 0.9855 - val_loss: 1.8280 - val_accuracy: 0.7394
Epoch 31/100
301/301 [==============================] - ETA: 0s - loss: 0.0522 - accuracy: 0.9837
Epoch 31: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 29ms/step - loss: 0.0522 - accuracy: 0.9837 - val_loss: 2.6189 - val_accuracy: 0.7200
Epoch 32/100
299/301 [============================>.] - ETA: 0s - loss: 0.0598 - accuracy: 0.9798
Epoch 32: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 29ms/step - loss: 0.0595 - accuracy: 0.9799 - val_loss: 2.1615 - val_accuracy: 0.7105
Epoch 33/100
299/301 [============================>.] - ETA: 0s - loss: 0.0506 - accuracy: 0.9869
Epoch 33: val_loss did not improve from 1.34885
301/301 [===========

301/301 [==============================] - 9s 29ms/step - loss: 0.0302 - accuracy: 0.9900 - val_loss: 2.1481 - val_accuracy: 0.7440
Epoch 60/100
299/301 [============================>.] - ETA: 0s - loss: 0.0298 - accuracy: 0.9902
Epoch 60: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 29ms/step - loss: 0.0297 - accuracy: 0.9902 - val_loss: 2.3331 - val_accuracy: 0.7259
Epoch 61/100
299/301 [============================>.] - ETA: 0s - loss: 0.0175 - accuracy: 0.9953
Epoch 61: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 29ms/step - loss: 0.0174 - accuracy: 0.9953 - val_loss: 2.1659 - val_accuracy: 0.7440
Epoch 62/100
299/301 [============================>.] - ETA: 0s - loss: 0.0266 - accuracy: 0.9915
Epoch 62: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 30ms/step - loss: 0.0264 - accuracy: 0.9916 - val_loss: 2.4899 - val_accuracy: 0.7343
Epoch 63/100
299/301 [============

Epoch 89/100
301/301 [==============================] - ETA: 0s - loss: 0.0174 - accuracy: 0.9956
Epoch 89: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 29ms/step - loss: 0.0174 - accuracy: 0.9956 - val_loss: 2.5929 - val_accuracy: 0.7386
Epoch 90/100
299/301 [============================>.] - ETA: 0s - loss: 0.0141 - accuracy: 0.9959
Epoch 90: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 29ms/step - loss: 0.0144 - accuracy: 0.9957 - val_loss: 2.8503 - val_accuracy: 0.7438
Epoch 91/100
301/301 [==============================] - ETA: 0s - loss: 0.0195 - accuracy: 0.9947
Epoch 91: val_loss did not improve from 1.34885
301/301 [==============================] - 9s 29ms/step - loss: 0.0195 - accuracy: 0.9947 - val_loss: 2.6333 - val_accuracy: 0.7389
Epoch 92/100
299/301 [============================>.] - ETA: 0s - loss: 0.0198 - accuracy: 0.9945
Epoch 92: val_loss did not improve from 1.34885
301/301 [===========

In [ ]:
#total sequence = 100
#max_sequence = 2
#head=1
#dense_dim=4 68.59
#128 = 68.89

#head=4
#128 = 74.73
#256 = 75.19

#head=6
#64 = 74.65
#128 = 75.78


#max_sequence = 10
#head=6
#4 = 75.65
#8 = 76.14
#16 = 76.33 
#32 = 76.65
#64 = 76.54 

In [15]:
# # Label preprocessing with StringLookup.
# label_processor = keras.layers.StringLookup(
#     num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
# )

# def get_compiled_model():
#     sequence_length = MAX_SEQ_LENGTH
#     embed_dim = NUM_FEATURES
#     dense_dim = 8
#     num_heads = 6
#     classes = len(label_processor.get_vocabulary())

#     inputs = keras.Input(shape=(None, None))
#     x = PositionalEmbedding(
#         sequence_length, embed_dim, name="frame_position_embedding"
#     )(inputs)
# #     x = TransformerEncoder2(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
# #     x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
# #     x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer2")(x)
# #     x = TransformerEncoder3(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)

# #     attention_output = keras.layers.MultiHeadAttention(
# #         num_heads=num_heads, key_dim=embed_dim, dropout=0.1
# #     )(x, x)

# #     proj_input = keras.layers.LayerNormalization()(x + attention_output)
# #     proj_output = keras.Sequential(
# #         [keras.layers.Dense(dense_dim, activation=tf.nn.gelu), keras.layers.Dense(embed_dim),]
# #     ) (proj_input)
# #     representation = keras.layers.LayerNormalization()(proj_input + proj_output)

#     # Layer normalization and MHSA
#     encoded_patches = x
    
#     for _ in range(2):
# #         x1 = keras.layers.LayerNormalization()(encoded_patches)
#         x1 = encoded_patches
#         attention_output = keras.layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=embed_dim, dropout=0.1
#         )(x1, x1)

#         # Skip connection
#         x2 = keras.layers.Add()([attention_output, x1])

#         # Layer Normalization and MLP
#         x3 = keras.layers.LayerNormalization()(x2)
#         x3 = keras.Sequential(
#             [
#                 keras.layers.Dense(units=dense_dim, activation=tf.nn.gelu),
#                 keras.layers.Dense(units=embed_dim),
#             ]
#         )(x3)

#         # Skip connection
#         encoded_patches = keras.layers.Add()([x3, x2])
#         encoded_patches = keras.layers.LayerNormalization()(encoded_patches)

#     representation = keras.layers.GlobalMaxPooling1D()(encoded_patches)
#     representation = keras.layers.Dropout(0.5)(representation)
    
#     outputs = keras.layers.Dense(classes, activation="softmax")(representation)
#     model = keras.Model(inputs, outputs)

#     model.compile(
#         optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
#     )
#     return model


# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     model = get_compiled_model()
#     history = model.fit(
#         train_data,
#         train_labels,
#         # validation_split=0.15,
#         validation_data=([test_data, test_labels]),
#         epochs=200,
#         callbacks=[checkpoint],
#     )
    
#     print(f"Test best accuracy: {round(max(history.history['val_accuracy'])*100,2)}%")

# #     model.load_weights(filepath)
# #     _, accuracy = model.evaluate(test_data, test_labels)
# #     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return model

# trained_model = run_experiment()

2023-01-21 17:05:32.627501: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 781271040 exceeds 10% of free system memory.
2023-01-21 17:05:33.135820: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 781271040 exceeds 10% of free system memory.


Epoch 1/200
 73/299 [======>.......................] - ETA: 20s - loss: 7.0645 - accuracy: 0.0090

KeyboardInterrupt: 

In [ ]:
#total sequence = 100
#max_sequence = 4
#head=1
#dense_dim=4 69.26
#8 71.50
#16 70.82
#32 68.70
#64 71.21
#128 69.65
#256
#512
#1024


#max_sequence = 2
#head=1
#dense_dim=4 71.08
#128 70.05
#head=4
#128 75.02
#256 74.76
#head=6
#128 75.57
#64 75.39

##max_sequence = 10
#head=6
#64 77.24
#128 76.53
#32 77.4
#1024 74.81
#16 77.13
#8 76.55

##max_sequence =20
#head=6
#8 77.08
#16 76.55
#4 75.73
#32 77.24
#64 75.84